### Scenario 2: A cross-functional team with 1 data scientist working on an ML Model

MLFlow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following conmmand in your terminal:

`mlflow server -- backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow 

mlflow.set_tracking_uri("http://localhost:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}")

tracking URI: 'http://localhost:5000


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747437551019, experiment_id='0', last_update_time=1747437551019, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score 

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y) 
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'") 

2025/05/16 19:21:13 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/16 19:21:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/5d920b95252d487a8f52e15f8d1795f5/artifacts'
🏃 View run fearless-ant-637 at: http://localhost:5000/#/experiments/1/runs/5d920b95252d487a8f52e15f8d1795f5
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747437673758, experiment_id='1', last_update_time=1747437673758, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747437551019, experiment_id='0', last_update_time=1747437551019, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with model registry

In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri="http://localhost:5000")

In [7]:
client.search_registered_models()

[]

In [20]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier',
)

Successfully registered model 'iris-classifier'.
2025/05/16 19:26:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1747437984451, current_stage='None', description='', last_updated_timestamp=1747437984451, name='iris-classifier', run_id='5d920b95252d487a8f52e15f8d1795f5', run_link='', source='mlflow-artifacts:/1/5d920b95252d487a8f52e15f8d1795f5/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>